In [1]:
import pandas
import matplotlib.pyplot as plt
import seaborn 
import gzip

%matplotlib inline

In [7]:
with gzip.GzipFile('../data/exports_cran.csv.gz') as f:
    exports = pandas.read_csv(f, usecols=['repository', 'ref', 'export'])
with gzip.GzipFile('../data/index_cran.csv.gz') as f: 
    packages = pandas.read_csv(f, parse_dates=['time'], usecols=['source', 'repository', 'ref', 'time'])

In [8]:
exports = exports.rename(columns={'repository': 'package', 'ref': 'version', 'export': 'item'})
packages = packages.query('source == "cran"').rename(columns={'repository': 'package', 'ref': 'version'})

In [9]:
packages = (packages
    .groupby(by='package')
    .time
    .agg({'pkg_firstdate': pandas.np.min, 'pkg_lastdate': pandas.np.max})
    .merge(packages, how='left', left_index=True, right_on='package')
    .drop('source', axis=1)
)

exports = exports.merge(packages[['package', 'version', 'time']], on=('package', 'version')).sort_values('time')

In [30]:
by_item = {}
by_version = {}

for package, package_group in exports.groupby('package'):
    first_version = True
    last_items = set()
        
    for version, version_group in package_group.groupby('time'):        
        items = set(version_group.item)    
        additions = items.difference(last_items)
        deletions = last_items.difference(items)
        last_items = set(items)
        
        for item in items:
            by_item.setdefault((package, item), {'deleted_in': pandas.NaT})
        for item in additions:
            by_item[(package, item)]['added_in'] = version
        for item in deletions:
            by_item[(package, item)]['deleted_in'] = version
    
        by_version[(package, version)] = {'items': len(items), 
                                          'additions': len(additions), 
                                          'deletions': len(deletions)}

KeyboardInterrupt: 

In [29]:
df_by_item = pandas.DataFrame.from_dict(by_item, orient='index')
df_by_version = pandas.DataFrame.from_dict(by_version, orient='index')

In [26]:
df_by_item.dropna()

deleted_in added_in
ACEt       AtCtEp                         1.5.3      1.3
           AtEtp                          1.5.3      1.3
ADaCGH     pSegmentHaarSeg                1.3-4    1.5-3
           tempdir2                       1.3-4    1.5-3
AER        nobs.survreg                   1.2-0    1.1-9
AGSDest    AGSTobj                          2.2      1.0
AICcmodavg AICc.clm                       2.0-1     2.00
           AICc.clmm                      2.0-1     2.00
           AICc.coxph                     2.0-1     2.00
           AICc.glm                       2.0-1     2.00
           AICc.gls                       2.0-1     2.00
           AICc.lme                       2.0-1     2.00
           AICc.mer                       2.0-1     2.00
           AICc.merMod                    2.0-1     2.00
           AICc.nlme                      2.0-1     2.00
           AICc.nls                       2.0-1     2.00
           AICc.polr                      2.0-1     2.00
           AICc.rlm                       2.0-1     2.00
           AICcCustom                      2.00    2.0-1
           DIC.bugs                       2.0-1     2.00
           DIC.rjags                      2.0-1     2.00
           aictabCustom                    2.00    2.0-1
           countDist                       2.00    2.0-4
           countHist                       2.00    2.0-4
           covDiag                         2.00    2.0-4
           detHist                         2.00    2.0-4
           extractCN                       2.00    2.0-1
           extractSE.mer                  2.0-1     2.00
           extractSE.merMod               2.0-1     2.00
           modavgCustom                    2.00    2.0-1
...                                         ...      ...
zendeskR   getOrganizations                 0.3      0.1
           getTickets                       0.3      0.1
           getTicketsForOrganization        0.4      0.3
zic        zic.ssvs                       0.7.5    0.5-2
zipfR      plot.spc                       0.6-6    0.6-0
           plot.vgc                       0.6-6    0.6-0
           zipfR.legend                   0.6-6    0.6-0
zoo        as.Date.numeric                1.7-2    1.7-5
           as.Date.ts                     1.7-2    1.7-5
           as.Date.yearmon                1.7-2    1.7-5
           as.Date.yearqtr                1.7-2    1.7-5
           as.yearqtr.yearqtr            1.7-10    1.7-2
           autoplot.zoo                   1.7-2    1.7-9
           facet_free                     1.7-2    1.7-9
           fortify.zoo                    1.7-2    1.7-9
           panel.arrows.its               1.6-3    1.2-0
           panel.arrows.tis               1.6-3    1.5-7
           panel.arrows.ts                1.6-3    1.2-0
           panel.arrows.zoo               1.6-3    1.2-0
           rapply                         1.4-0    1.1-0
           rollsum                        1.7-2    1.7-8
           rollsum.default                1.7-2    1.7-8
           rollsumr                       1.7-2    1.7-8
           scale_x_yearmon                1.7-2   1.7-10
           scale_x_yearqtr                1.7-2   1.7-10
           scale_y_yearmon                1.7-2   1.7-10
           scale_y_yearqtr                1.7-2   1.7-10
           yearmon_trans                  1.7-2   1.7-10
           yearqtr_trans                  1.7-2   1.7-10
ztable     myhtmlHead                     0.1.3    0.1.0

[33803 rows x 2 columns]

# API changes

## Changes distribution by packages and by versions

##  Changes by dates

## Changes by ages

# API changes correlation

## For packages

## For versions

# Survival analysis

## Deletions since introduction